In [3]:
def levenshtein_distance(s1, s2):
    # Erstelle eine Matrix mit Dimensionen (len(s1)+1) x (len(s2)+1)
    matrix = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]

    # Initialisiere die erste Zeile und Spalte
    for i in range(len(s1) + 1):
        matrix[i][0] = i
    for j in range(len(s2) + 1):
        matrix[0][j] = j

    # Fülle die Matrix
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            # Kosten je gleichheit
            cost = 0 if s1[i-1] == s2[j-1] else 1
            matrix[i][j] = min(
                matrix[i-1][j] + 1,
                matrix[i][j-1] + 1,
                matrix[i-1][j-1] + cost
            )

    return matrix[len(s1)][len(s2)]

print(levenshtein_distance("TISCHLER", "FISCHER"))

2


In [5]:
def autocorrect(word, dictionary, max_distance=2):
    """Gibt Korrekturvorschläge für ein falsch geschriebenes Wort zurück."""
    suggestions = []

    for dict_word in dictionary:
        distance = levenshtein_distance(word, dict_word)
        if distance <= max_distance:
            suggestions.append((dict_word, distance))

    # Sortiere nach Distanz (kleinere zuerst)
    return sorted(suggestions, key=lambda x: x[1])

# Beispiel-Wörterbuch
german_words = ["Haus", "Maus", "Laus", "Baum", "Raum", "Traum", "Schaum"]

# Anwendung
typo = "Huas"
corrections = autocorrect(typo, german_words)
print(f"Meintest du vielleicht: {[word for word, _ in corrections]}")

Meintest du vielleicht: ['Haus']


In [6]:
def levenshtein_distance_optimized(s1, s2):
    # Stelle sicher, dass s1 der kürzere String ist
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    # Speichere nur die aktuelle und die vorherige Zeile
    previous_row = list(range(len(s2) + 1))
    current_row = [0] * (len(s2) + 1)

    for i in range(1, len(s1) + 1):
        current_row[0] = i

        for j in range(1, len(s2) + 1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            current_row[j] = min(
                previous_row[j] + 1,      # Löschen
                current_row[j-1] + 1,     # Einfügen
                previous_row[j-1] + cost  # Ersetzen oder übereinstimmen
            )

        # Tausche die Zeilen für die nächste Iteration
        previous_row, current_row = current_row, previous_row

    return previous_row[len(s2)]

print(levenshtein_distance("TISCHLER", "FISCHER"))

2


In [7]:
def find_similar_products(query, product_catalog, threshold=0.8):
    """Findet ähnliche Produkte basierend auf der Levenshtein-Distanz."""
    suggestions = []

    for product_name, product_info in product_catalog.items():
        # Normalisierte Distanz berechnen (0 bis 1, wobei 1 identisch ist)
        max_len = max(len(query), len(product_name))
        if max_len == 0:
            similarity = 1.0
        else:
            distance = levenshtein_distance_optimized(query, product_name)
            similarity = 1.0 - distance / max_len

        if similarity >= threshold:
            suggestions.append((product_name, product_info, similarity))

    # Sortiere nach Ähnlichkeit (höhere zuerst)
    return sorted(suggestions, key=lambda x: x[2], reverse=True)

# Beispiel-Produktkatalog
products = {
    "Lederjacke schwarz": {"preis": 129.99, "kategorie": "Kleidung"},
    "Lederjacke braun": {"preis": 139.99, "kategorie": "Kleidung"},
    "Lederweste schwarz": {"preis": 89.99, "kategorie": "Kleidung"},
    "Lederschuhe schwarz": {"preis": 99.99, "kategorie": "Schuhe"},
}

# Anwendung
search_query = "Lederjacke shwarz"  # Tippfehler
similar_products = find_similar_products(search_query, products)
print("Ähnliche Produkte:")
for name, info, similarity in similar_products:
    print(f"- {name} (Ähnlichkeit: {similarity:.2f})")

Ähnliche Produkte:
- Lederjacke schwarz (Ähnlichkeit: 0.94)
